In [1]:
import os
import tensorflow as tf

In [5]:
def read_and_decode(filename_queue, batch_size):
    tf_record_reader=tf.TFRecordReader()
    _, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

    tf_record_features = tf.parse_single_example(
        tf_record_serialized,
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string),
        })
    
    tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)
    
    tf_record_image = tf.cast(tf_record_image, tf.float32)

    tf_record_image = tf.reshape(tf_record_image, [64,64])

    tf_record_label = tf.cast(tf_record_features['label'],tf.int64)
    


    images, labels = tf.train.shuffle_batch([tf_record_image, tf_record_label],
                                         batch_size = batch_size,
                                         capacity= 20*batch_size,
                                         min_after_dequeue= 5*batch_size )
    return images, labels

In [3]:
file_name = "int_HanDB_Test.tfrecord"
tf_record_filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("./"+file_name))
#f=open("./test_record.tfrecord",'rb')

In [4]:
tf_record_reader=tf.TFRecordReader()
_, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

In [5]:
tf_record_features = tf.parse_single_example(
    tf_record_serialized,
    features={
        'label': tf.FixedLenFeature([], tf.string),
        'width': tf.FixedLenFeature([], tf.int64),
        'height': tf.FixedLenFeature([], tf.int64),
        'image': tf.FixedLenFeature([], tf.string),
    })

In [7]:
#tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)

#tf_record_image = tf.reshape(tf_record_image, [64,64,1])

#tf_record_label = tf.cast(tf_record_features['label'],tf.string)

images, labels = read_and_decode(tf_record_filename_queue,10)

init=tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())

record_iterator = tf.python_io.tf_record_iterator(path=file_name)

c=0
for record in record_iterator:
    c+=1
print(c)

with tf.Session() as sess:
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for i in range(3):
        img, lab = sess.run([images,labels])
        print(lab)
    coord.request_stop()
    coord.join(threads)



75186
[48879 49405 45500 46052 46064 47050 46820 48818 46067 51143]
[47286 46068 48881 47564 46287 47797 48579 46030 51454 49135]
[48087 48545 46519 45274 45816 46518 48103 48316 50385 49363]
